In [ ]:
import os
import json
import csv
from typing import List

In [ ]:
def write_json_file(data: dict, filename: str) -> None:
  with open(filename, 'w') as file:
    json.dump(data, file)

In [ ]:
def read_json_file(filename: str) -> dict:
  with open(filename, 'r') as file:
    return json.load(file)

In [ ]:
def write_csv(data: List[dict], filename: str) -> None:
  headers = list(data[0].keys())
  with open(filename, 'w') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(data)

# PACE CLI Example Usage

## Displaying Help

In [ ]:
!pace-cli -h

**Note:** Help can also be displayed for subcommands

In [ ]:
!pace-cli package -h

In [ ]:
!pace-cli package process -h

## Displaying Version

In [ ]:
!pace-cli -V

**Note:** Version can be displayed for subcommands

In [ ]:
!pace-cli package -V

## Interacting with Metadata
### Table of Contents:
- [Create Object](#Create-Object)
  - [Create from File](#Create-from-File)
  - [Create from Standard Input](#Create-from-Standard-Input)
- [Read Objects](#Read-Objects)
  - [List Objects](#List-Objects)
  - [Get Object by Uniquely Identifying Field](#Get-Object-by-Uniquely-Identifying-Field)
  - [Get Object by UUID](#Get-Object-by-UUID)
- [Update Object](#Update-Object)
  - [Update from File](#Update-from-File)
  - [Update from Standard Input](#Update-from-Standard-Input)
- [Delete Object](#Delete-Object)
  - [Delete by UUID](#Delete-by-UUID)
- [Translating Objects](#Translating-Objects)

### Create Object

In [ ]:
example_person = {
  "name": "example-person",
  "position": "scientist",
  "organization": "CIRES",
  "street": "216 UCB",
  "city": "Boulder",
  "state": "Colorado",
  "zip": "80309",
  "phone": "123-456-7890",
}

write_json_file(example_person, 'person1.json')

#### Create from File

In [ ]:
!pace-cli person create person1.json 

#### Create from Standard Input

In [ ]:
example_person = {
  "name": "another-person",
  "position": "scientist",
  "organization": "CIRES",
  "street": "216 UCB",
  "city": "Boulder",
  "state": "Colorado",
  "zip": "80309",
  "phone": "123-456-7890",
}

write_json_file(example_person, 'person2.json')

In [ ]:
!cat person2.json | pace-cli person create - 

### Read Objects

#### List Objects

In [ ]:
!pace-cli person list

**Note:** objects can be filtered to constrain list results

In [ ]:
!pace-cli person list --names another,example

#### Get Object by Uniquely Identifying Field

In [ ]:
!pace-cli person get-by-name another-person

#### Get Object by UUID

In [ ]:
# retrieving uuid from previous result
!pace-cli person get-by-name another-person > result.json

# extracting uuid from file
example_person = read_json_file('result.json')
uuid = example_person.get('uuid')

# query by uuid
!pace-cli person get-by-uuid {uuid}

### Update Object

#### Update from File

In [ ]:
example_person['name'] = "edited-person-name"
write_json_file(example_person, 'person1.json')

!pace-cli person update person1.json

#### Update from Standard Input

In [ ]:
example_person['name'] = 'another-edited-person-name'
write_json_file(example_person, 'person1.json')

!cat person1.json | pace-cli person update -

### Delete Object

#### Delete by UUID

In [ ]:
!pace-cli person delete {uuid}

### Translating Objects

Create translator:

In [ ]:
example_translator = {
  "type": "person",
  "name": "people-translator",
  "person_uuid": "UUID",
  "person_name": "NAME",
  "position": "POSITION",
  "organization": "ORGANIZATION",
  "street": "STREET",
  "city": "CITY",
  "state": "STATE",
  "zip": "ZIP",
  "country": "COUNTRY",
  "phone": "PHONE",
  "email": "EMAIL",
  "orcid": "ORCID"
}

write_json_file(example_translator, 'person-translator.json')

In [ ]:
!pace-cli translator create person-translator.json

Write CSV file:

In [ ]:
person1 = {
  "UUID": "",
  "NAME": "person-1",
  "POSITION": "scientist",
  "ORGANIZATION": "CIRES",
  "STREET": "216 UCB",
  "CITY": "Boulder",
  "STATE": "Colorado",
  "ZIP": "80309",
  "COUNTRY": "USA",
  "PHONE": "123-456-7890",
  "EMAIL": "",
  "ORCID": ""
}

person2 = {
  "UUID": "",
  "NAME": "person-2",
  "POSITION": "scientist",
  "ORGANIZATION": "CIRES",
  "STREET": "216 UCB",
  "CITY": "Boulder",
  "STATE": "Colorado",
  "ZIP": "80309",
  "COUNTRY": "USA",
  "PHONE": "123-456-7890",
  "EMAIL": "",
  "ORCID": ""
}

write_csv([ person1, person2 ], 'people.csv')

!cat people.csv

Translate objects from CSV file: 

In [ ]:
!pace-cli person translate -tf csv -tn people-translator people.csv

**Note:** Translating an object does not save it to PACE. To save, use your output as a parameter to the ``create`` command 

In [ ]:
!pace-cli person translate -tf csv -tn people-translator people.csv | pace-cli person create -

## Interacting with Packages

### Create Package

#### Create from File

Sea areas not provided in default metadata. Create example sea area: 

In [ ]:
example_sea_area = { 'name': 'Pacific' }

write_json_file(example_sea_area, 'sea.json')

!pace-cli sea create sea.json

Create package:

In [ ]:
example_package = {
  'type': 'audio',
  'data_collection_name': 'Example Package 1',
  'source_path': '/Users/user/data-dir/audio-data',
  'dataset_packager': 'Chuck Anderson',
  'sponsors': [ 'NOAA AFSC' ],
  'funders': [ 'NOAA AFSC' ],
  'public_release_date': '2024-08-30',
  'platform': 'Mooring',
  'instrument': 'SoundTrap 500',
  'quality_analyst': 'Chuck Anderson',
  'location_detail': {
    'type': 'stationary marine',
    'sea_area': 'Pacific',
    'recovery_location': {
      'latitude': 1.5,
      'longitude': 2.1,
      'instrument_depth': 1000.1
    },
    'deployment_location': {
      'latitude': 3.7,
      'longitude': 4.3,
      'instrument_depth': 9999.2
    }
  },
  'channels': [
    {
      'start_time': '2024-08-30T10:00:00Z',
      'end_time': '2024-08-30T12:00:00Z',
      'sample_rates': [
        {}
      ],
      'duty_cycles': [],
      'gains': []
    }
  ]
}

write_json_file(example_package, 'package1.json')

!pace-cli package create package1.json

#### Create from Standard Input

In [ ]:
example_package = {
  'type': 'audio',
  'data_collection_name': 'Example Package 2',
  'source_path': '/Users/user/data-dir/audio-data',
  'dataset_packager': 'Chuck Anderson',
  'sponsors': [ 'NOAA AFSC' ],
  'funders': [ 'NOAA AFSC' ],
  'public_release_date': '2024-08-30',
  'platform': 'Mooring',
  'instrument': 'SoundTrap 500',
  'quality_analyst': 'Chuck Anderson',
  'location_detail': {
    'type': 'stationary marine',
    'sea_area': 'Pacific',
    'recovery_location': {
      'latitude': 1.5,
      'longitude': 2.1,
      'instrument_depth': 1000.1
    },
    'deployment_location': {
      'latitude': 3.7,
      'longitude': 4.3,
      'instrument_depth': 9999.2
    }
  },
  'channels': [
    {
      'start_time': '2024-08-30T10:00:00Z',
      'end_time': '2024-08-30T12:00:00Z',
      'sample_rates': [],
      'duty_cycles': [],
      'gains': []
    }
  ]
}

write_json_file(example_package, 'package2.json')

!cat package2.json | pace-cli package create -

### Read Packages

#### List Packages

In [ ]:
!pace-cli package list

**Note:** packages can be filtered to constrain list results

In [ ]:
!pace-cli package list --package-ids "Example Package 1"

#### Get Package by Package ID

In [ ]:
!pace-cli package get-by-package-id "Example Package 1"

#### Get Package by UUID

In [ ]:
# retrieving uuid from previous result
!pace-cli package get-by-package-id "Example Package 1" > result.json

# extracting uuid from file
example_package = read_json_file('result.json')
package_uuid = example_package.get('uuid')

# query by uuid
!pace-cli package get-by-uuid {package_uuid}

### Update Package

#### Update from File

In [ ]:
example_package['data_collection_name'] = 'Edited Package 1'
write_json_file(example_package, 'package.json')

!pace-cli package update package.json

#### Update from Standard Input

In [ ]:
example_package['data_collection_name'] = 'Edited Package 1'
write_json_file(example_package, 'package.json')

!cat package.json | pace-cli package update -

### Delete Package

#### Delete by UUID

In [ ]:
!pace-cli package delete {package_uuid} 

### Translating Packages  

Create translator:

In [ ]:
audio_package_translator = {
  'type': 'audio package',
  'name': 'audio package translator',
  'data_collection_name': 'DATA_COLLECTION_NAME',
  'source_path': 'SOURCE_PATH',
  'dataset_packager': 'DATASET_PACKAGER',
  'sponsors': 'SPONSORS',
  'funders': 'FUNDERS',
  'public_release_date': {
    'date': 'PUBLIC_RELEASE_DATE',
    'time_zone': 'TIME_ZONE'
  },
  'platform': 'PLATFORM',
  'instrument': 'INSTRUMENT',
  'quality_control_detail_translator': {
    'quality_analyst': 'QUALITY_ANALYST'
  },
  'location_detail_translator': {
    'type': 'stationary marine',
    'sea_area': 'SEA_AREA',
    'deployment_location_translator': {
      'latitude': 'DEPLOYMENT_LATITUDE',
      'longitude': 'DEPLOYMENT_LONGITUDE',
      'instrument_depth': 'DEPLOYMENT_INSTRUMENT_DEPTH'
    },
    'recovery_location_translator': {
      'latitude': 'RECOVERY_LATITUDE',
      'longitude': 'RECOVERY_LONGITUDE',
      'instrument_depth': 'RECOVERY_INSTRUMENT_DEPTH'
    },
  },
  'channel_translators': [
    {
      'start_time': {
        'type': 'default',
        'time_zone': 'TIME_ZONE',
        'time': 'CHANNEL_START_TIME'
      },
      'end_time': {
        'type': 'default',
        'time_zone': 'TIME_ZONE',
        'time': 'CHANNEL_END_TIME'
      }
    }
  ]
}

write_json_file(audio_package_translator, 'package-translator.json')

!pace-cli translator create package-translator.json

Write CSV file:

In [ ]:
audio_package_translator.keys()

In [ ]:
package1 = {
  'DATA_COLLECTION_NAME': 'Translated Package 1',
  'SOURCE_PATH': '/Users/user/data-dir/audio-data',
  'DATASET_PACKAGER': 'Chuck Anderson',
  'SPONSORS': 'NOAA AFSC',
  'FUNDERS': 'NOAA AFSC',
  'TIME_ZONE': 'UTC',
  'PUBLIC_RELEASE_DATE': '2024-08-30',
  'PLATFORM': 'Mooring',
  'INSTRUMENT': 'SoundTrap 500',
  'QUALITY_ANALYST': 'Chuck Anderson',
  'SEA_AREA': 'Pacific',
  'DEPLOYMENT_LATITUDE': 1.2,
  'DEPLOYMENT_LONGITUDE': 99.1,
  'DEPLOYMENT_INSTRUMENT_DEPTH': 1000.1,
  'RECOVERY_LATITUDE': 2.2,
  'RECOVERY_LONGITUDE': 89.1,
  'RECOVERY_INSTRUMENT_DEPTH': 1001.2,
  'CHANNEL_START_TIME': '2024-08-30T12:00:00',
  'CHANNEL_END_TIME': '2024-08-30T14:00:00'
}

package2 = {
  'DATA_COLLECTION_NAME': 'Translated Package 2',
  'SOURCE_PATH': '/Users/user/data-dir/audio-data',
  'DATASET_PACKAGER': 'Chuck Anderson',
  'SPONSORS': 'NOAA AFSC',
  'FUNDERS': 'NOAA AFSC',
  'TIME_ZONE': 'UTC',
  'PUBLIC_RELEASE_DATE': '2024-08-31',
  'PLATFORM': 'Mooring',
  'INSTRUMENT': 'SoundTrap 500',
  'QUALITY_ANALYST': 'Chuck Anderson',
  'SEA_AREA': 'Pacific',
  'DEPLOYMENT_LATITUDE': 1.9,
  'DEPLOYMENT_LONGITUDE': 92.1,
  'DEPLOYMENT_INSTRUMENT_DEPTH': 1200.3,
  'RECOVERY_LATITUDE': 2.3,
  'RECOVERY_LONGITUDE': 81.1,
  'RECOVERY_INSTRUMENT_DEPTH': 1206.9,
  'CHANNEL_START_TIME': '2024-08-31T12:00:00',
  'CHANNEL_END_TIME': '2024-08-31T14:00:00'
}

write_csv([ package1, package2 ], 'packages.csv')

!cat packages.csv

Translate packages from CSV file:

In [ ]:
!pace-cli package translate -tf csv -tn "audio package translator" packages.csv

**Note:** Translating a package does not save it to PACE. To save, use your output as a parameter to the ``create`` command

In [ ]:
!pace-cli package translate -tf csv -tn "audio package translator" packages.csv | pace-cli package create -

### Processing Packages

Create data to process (demo purposes only):

In [ ]:
# get package from previous result
!pace-cli package get-by-package-id "Translated Package 1" > package.json
example_package = read_json_file('package.json')

# write mocked data to source path
source_path = f'{os.getcwd()}/audio-data'
os.mkdir(source_path)
example_package['source_path'] = source_path
for i in range(0, 1000):
  write_json_file({ 'field-1': f'value-{i}' }, f'{source_path}/file-{i}.json')

# save package changes
write_json_file(example_package, 'package.json')
!pace-cli package update package.json

#### Process from File

In [ ]:
!pace-cli package process package.json .

#### Process from Standard Input

In [ ]:
!rm -rf "Translated Package 1"
!cat package.json | pace-cli package process - .

**Note:** Logs are written to standard error. A progress bar is written to standard output. These outputs can be separated.

In [ ]:
# show standard error only
!rm -rf "Translated Package 1"
!pace-cli package process package.json . 1> /dev/null

In [ ]:
# show standard output only
!rm -rf "Translated Package 1"
!pace-cli package process package.json . 2> /dev/null